In [7]:
#Libraries includig keras, numpy and matplot etc

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

from numpy import mean
from numpy import std
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")

from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
import os

from keras.losses import binary_crossentropy
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
#from keras.optimizers import SGD
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot

In [8]:
#mounting drive 
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
#Getting the images you can get them through the link in report
data = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/Project2/Aug",
    labels='inferred',label_mode='binary',image_size=(128,128),
    batch_size=1,shuffle=True,seed=5
    )


Found 69 files belonging to 2 classes.


In [10]:
class_names = data.class_names


In [11]:
#Creating Inputs and Labels
dataX = []
dataY = []
for i in data.as_numpy_iterator():
  dataX.append(i[0])
  dataY.append(i[1])
dataX = np.array(dataX)
dataY = np.array(dataY)


In [12]:
#load dataset and return train values
def load_dataset(): 
  trainX, trainY= dataX,dataY
  # reshape dataset to have a single channel
  trainX = trainX.reshape((trainX.shape[0],128,128,3))

  return trainX, trainY


In [13]:
def prep_pixels(train):
  # convert from integers to floats
  train_norm = train.astype('float32')
  # normalize to range 0-1
  train_norm = train_norm / 255.0
  # return normalized images
  return train_norm

In [15]:
#Defining/Making a model with multiple convultional and Maxpooling layers then flatten it to use in our main model
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(128,128,3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(1, activation='sigmoid'))
  # compile model
  model.compile(optimizer=Adam(lr=0.001), loss=binary_crossentropy, metrics=['accuracy'])
  model.summary()

  return model




In [45]:
# evaluate a model using k-fold cross-validation
#It all find F1,recalls and other features for each fold
def train_and_evaluate_model(dataX, dataY, n_folds=3):
  train_accuracies = []
  val_accuracies = []
  f1_scores = []
  precisions = []
  recalls = []
  roc_aucs = []
  scores, histories = list(), list()
  # prepare cross validation
  kfold = KFold(n_folds, shuffle=True, random_state=4)
  # enumerate splits

  for train_ix, test_ix in kfold.split(dataX):
    # define model
    model = define_model()
    # select rows for train and test
    trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
    # fit model
    history = model.fit(trainX, trainY, epochs=10, batch_size=10, validation_data=(testX, testY), verbose=1)
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=1)
    print('> %.3f' % (acc * 100.0))
    # stores scores
    scores.append(acc)

        # Fit the model on the current training data
   
        # Get the training and validation accuracy
    train_acc = model.evaluate(trainX, trainY, verbose=0)[1]
    val_acc = model.evaluate(testX, testY, verbose=0)[1]
        # Get the predictions on the validation data
    val_preds = model.predict(testX)
    val_preds = np.round(val_preds)
    testY = testY.reshape(-1)
    

        # Get the F1 score, precision, recall, and ROC AUC
    f1 = f1_score(testY, val_preds)
    precision = precision_score(testY, val_preds)
    recall = recall_score(testY, val_preds)
    roc_auc = roc_auc_score(testY, val_preds)


    print('f1 score: ',f1)
    print('Precision:', precision)
    print('Recall:', recall)
    print('AUC: ',roc_auc)


        # Append the metrics to the lists
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    f1_scores.append(f1)
    precisions.append(precision)
    recalls.append(recall)
    roc_aucs.append(roc_auc)

    # Get the test set accuracy
    histories.append(history)
    #Testtraingraph(history)
  return scores, histories,n_folds,model


# plot diagnostic learning curves
def summarize_diagnostics(histories):
  for i in range(len(histories)):
    # plot loss
    pyplot.subplot(2, 1, 1)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(histories[i].history['loss'], color='blue', label='train')
    pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(2, 1, 1)
    pyplot.title('Classification Accuracy')
    pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
    pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
  pyplot.show()


#training and validation graph per fold
def Testtraingraph(histories):
  # plot loss
    pyplot.subplot(2, 1, 1)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(histories.history['loss'], color='blue', label='train')
    pyplot.plot(histories.history['val_loss'], color='orange', label='test')
    pyplot.show()
    # plot accuracy
    pyplot.subplot(2, 1, 2)
    pyplot.title('Classification Accuracy')
    pyplot.plot(histories.history['accuracy'], color='blue', label='train')
    pyplot.plot(histories.history['val_accuracy'], color='orange', label='test')
    pyplot.show()

# summarize model performance
def summarize_performance(scores):
  # print summary
  print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
  # box and whisker plots of results
  pyplot.boxplot(scores)
  pyplot.show()



In [ ]:
#loading dataset
trainX,trainY=load_dataset()
#Normalizing./Rescaling between 0 and 1
trainX = prep_pixels(trainX)
#Validating our model
scores, histories,folds,model = train_and_evaluate_model(trainX,trainY)
# learning curves
summarize_diagnostics(histories)
# summarize estimated performance
summarize_performance(scores)

In [ ]:
#Validaton and train graphfinal
Totalloss=int(len(histories[0].history['loss']))
avg_loss=np.zeros(Totalloss)
avg_val_loss=np.zeros(Totalloss)
for i in range(folds):
  avg_loss=np.add(avg_loss,histories[i].history['loss'])
  avg_val_loss = np.add(histories[i].history['val_loss'],avg_val_loss)

avg_loss=avg_loss/folds
avg_val_loss=avg_val_loss/folds

plt.plot(avg_loss[:],c='r')
plt.plot(avg_val_loss[:],c='b')
plt.xlabel("Epoches")
plt.ylabel("Loss")
plt.legend(["Training Loss","Test Loss"])
plt.show()

In [47]:
#Give a picture and make test it if it is stroke or not stroke patient
def Prediction(file_id):
  # Import the necessary libraries
  import keras.utils as image
  from keras.models import load_model

  # Download the image from Google Drive using the file ID


  # Load the image and preprocess it for the model
  img = image.load_img(file_id, target_size=(128, 128))
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)

  # Load the pre-trained model
  pred = model.predict(img_array)

  # Get the class with the highest probability
  class_index = np.argmax(pred)
  # Print the accuracy of the model
  if(pred>0.5):
    print("Stroke","Class Index for this picture/Stroke Patient : " , class_index)
  else:
    print("Non-Stroke ","Class Index for this picture/Non-Stroke Patient : " ,class_index)
  return pred


In [46]:
#Give a image int the following variable File_id kindly put a link from your drive to run it
file_id = '/content/drive/MyDrive/Project2/Test/A3.jpg'
Prediction(file_id)

1/1 [==============================] - 0s 30ms/step
Non-Stroke  Class Index for this picture/Non-Stroke Patient :  0


array([[0.]], dtype=float32)